In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [12]:
K = 4
lambda_k_true = np.array([5,5,0,2])

I = 5
J = 50


sigma = 1
S = 2
### generate exogenous data
np.random.seed(1087)

# φ_i_j_k = np.random.normal(1,1, size=[I,J,K-1])
φ_i_j_k = (np.random.normal(0,1,size = [I])[:,None,None] 
            * np.random.normal(1,1, size=[J,K-1])[None,:,:])



eps_si_j = sigma * np.random.normal(0,1, size = [S*I,J])


def φ_k(i,B):
    # return φ_i_j_k[i,B,:].sum(0)
    return np.concatenate((φ_i_j_k[i,B,:].sum(0), [ - np.sum(B)**(1.2)]))

### greedy
def greedy(i,lambda_k, p_j = np.zeros(J)):
    B_k =  np.zeros(J, dtype=bool)
    val = np.inner(φ_k(i,B_k),lambda_k)
    k = 0 
    while k < J:
        j_k = None
        max_add_j =  - np.inf
        for j in np.where(1-B_k)[0]:
            val_add_j = np.inner(φ_k(i,B_k+ np.eye(1,J,j,dtype=bool)[0]), lambda_k)   - p_j[j] - p_j[B_k].sum() - val
            if val_add_j > max_add_j:
                j_k = j
                max_add_j = val_add_j
            
        if j_k is None or max_add_j < 0:
            break
        val += max_add_j 
        B_k[j_k] = 1
        k += 1
    return B_k , val

$\begin{align}
\min_{p}& \quad \frac{1}{S}\sum_{si} u_{si} + \sum_{j} p_j \notag\\
s.t.& \quad u_{si} + \sum_{j \in B}p_j \geq \phi_{iB}^\top\lambda + \epsilon_{siB}\quad \text{ for all }siB \notag
\end{align}$

In [13]:
max_iters = 3000
tol = 1e-12

constraints_list = []


data = []

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        u_si = model.addVars(S*I, name="utilities")
        # p_j = model.addVars(J, name="prices")
        p_j = model.addVars(J, lb= 0 ,ub=GRB.INFINITY, name="prices")

        # Set objective
        model.setObjective((1/S)* u_si.sum() + p_j.sum(), GRB.MINIMIZE)
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()
        
        # Extract the solution (theta_solution)
        theta_solution = np.array(model.x)

        
        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}")
            ### Pricing problem
            B_star_si = []
            val_si = []
            for si in range(S*I):
                        B_star, val = greedy(si //S ,lambda_k = lambda_k_true, p_j = theta_solution[-J:]- eps_si_j[si,:])
                        B_star_si.append(B_star)
                        val_si.append(val -  theta_solution[si])
            # stop if certificate holds
            print(f"reduced cost: {np.max(val_si)}")
            if np.max(val_si) <= tol:
                primal_solution = np.array(model.x)
                dual_solution = np.array(model.pi)
                print("DONE!")
                break
            
            ### Master problem
            model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])>= 
                            φ_k(si // S, B_star_si[si])@lambda_k_true + eps_si_j[si, B_star_si[si]].sum()
                            for si in range(S*I)), name="constraint_batch")
            # print number of  constrints of the model
            # print(f"Number of constraints: {len(model.getConstrs())}")
            constraints_list.append(B_star_si)

            # Optimize the model
            u_si.start = val_si + theta_solution[:S*I]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            # print(theta_solution.sum())
            print(model.ObjVal)
            data.append(model.ObjVal)


            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0
reduced cost: 830.5526908206024
837.678665470481
##############
ITER: 1
reduced cost: 778.150536284592
837.678665470481
##############
ITER: 2
reduced cost: 489.9822068205173
837.678665470481
##############
ITER: 3
reduced cost: 456.5790517816554
837.6786654704808
##############
ITER: 4
reduced cost: 472.82642237980133
837.6786654704808
##############
ITER: 5
reduced cost: 424.30602579243333
837.6786654704808
##############
ITER: 6
reduced cost: 426.9859301077272
837.6786654704808
##############
ITER: 7
reduced cost: 405.1626132099233
837.6786654704808
##############
ITER: 8
reduced cost: 464.0702138968318
837.6786654704808
##############
ITER: 9
reduced cost: 413.007751536314
837.678665470481
##############
ITER: 10
reduced cost: 453.8229066344011
837.678665470481
##############
ITER: 11
reduced cost: 45

In [17]:
np.all( (dual_solution == .5) | (dual_solution == 0) ) 

True

In [15]:
binding_constraints = np.array(constraints_list).reshape(-1,J)[dual_solution >0]
index = np.kron(np.ones(iter, dtype= int),np.arange(S*I))
matching = np.zeros((I * S,J), dtype=bool)
matching[index[dual_solution > 0]]=  binding_constraints

In [16]:
print(matching.sum(0))
print(matching.sum(1))

[2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 0 2 0 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 0 2
 2 2 2 2 2 2 0 2 2 2 2 2 2]
[ 0  0  3  3 37 41  0  0  2  4]


In [ ]:
np.random.seed(9989877)
I = np.maximum(1000, np.shape(matching)[0]/10)
random_draws = np.random.choice(np.arange(np.shape(matching)[0]), size= [I], replace=False)
sample = matching[random_draws,:]
draw_type = random_draws//S
m_j = sample.sum(0)
phi_k_hat = np.zeros(K)
for i in range(I):
    phi_k_hat += φ_k(draw_type[i],sample[i,:])
print(phi_k_hat)
print(m_j)
print(draw_type)

ValueError: Cannot take a larger sample than population when 'replace=False'

$\begin{align}
\max_{\lambda ,u,p}& \quad \sum_{ik} \phi_{i\hat B_i,k} \lambda_k - \frac{1}{S}\sum_{si} u_{si} -  \sum_{j} p_j \notag\\
s.t.& \quad u_{si} + \sum_{j \in B}p_j \geq \sum_{k} \phi_{i B,k} \lambda_k + \epsilon_{siB}\quad \text{ for all }siB \notag
\end{align}$

In [ ]:
max_iters = 100
tol = 1e-12

np.random.seed(443)
S = 50
eps_si_j = sigma * np.random.normal(0,1, size = [S*I,J])      

with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        lambda_k = model.addVars(K, lb= -GRB.INFINITY, ub = GRB.INFINITY , name="parameters")
        u_si = model.addVars(S*I, name="utilities")
        # p_j = model.addVars(J, name="prices")
        p_j = model.addVars(J,  lb= -GRB.INFINITY, ub = GRB.INFINITY , name="prices")

        # Set objective
        model.setObjective(gp.quicksum(phi_k_hat[k] * lambda_k[k] for k in range(K))
                            - (1/S)* u_si.sum() -  gp.quicksum(m_j[j] * p_j[j] for j in range(J)), GRB.MAXIMIZE)
        
        model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(sample[si // S])[0])>= 
                            gp.quicksum(φ_k(draw_type[si // S], sample[si // S])[k] *  lambda_k[k] for k in range(K))
                            + eps_si_j[si, sample[si // S]].sum()
                            for si in range(S*I)), name="constraint_batch")
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        # model.setParam('Presolve', 0)
        model.optimize()
        status = model.Status
        # Extract the solution 
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}, parameters: {theta_solution[:K]}")
            ### Pricing problem
            B_star_si = []
            val_si = []
            for si in range(S*I):
                        B_star, val = greedy(draw_type[si //S] , lambda_k = theta_solution[:K], p_j = theta_solution[-J:]- eps_si_j[si,:])
                        B_star_si.append(B_star)
                        val_si.append(val -  theta_solution[K+si])
            print(f"greedy done")

            # stop if certificate holds
            print(f"reduced cost: {np.max(val_si)}")
            if np.max(val_si) <= tol:
                primal_solutin = np.array(model.x)
                dual_solution = np.array(model.pi)
                break
            
            ### Master problem
            # add constraints
            model.addConstrs((u_si[si] + gp.quicksum(p_j[j] for j in np.where(B_star_si[si])[0])>= 
                            gp.quicksum(φ_k(draw_type[si // S], B_star_si[si])[k] *  lambda_k[k] for k in range(K))+ 
                            eps_si_j[si, B_star_si[si]].sum()
                            for si in range(S*I)), name="constraint_batch")
            
            # optimize the model
            lambda_k.start = theta_solution[:K]
            u_si.start = val_si + theta_solution[K:S*I +K]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0, parameters: [0.65603763 0.84187228 0.42259377 0.51986623]


KeyboardInterrupt: 

In [ ]:
print(lambda_k_true)
print(theta_solution[:K])

[ 3.   4.  -1.   1.5]
[ 3.02418866  4.07936004 -0.94168     1.60812049]
